In [ ]:
import pandas as pd
import featuretools as ft
import numpy as np
import math


Index = {
    (0 , 0): 0,
    (-1, 0): 1,
    (1 , 0): 2,
    (0 ,-1): 3,
    (0 , 1): 4
}

oppositeIndex = {
    (0 , 0): 0,
    (-1, 0): 2, 
    (1 , 0): 1,
    (0 ,-1): 4,
    (0 , 1): 3,
}


raw = pd.read_json("C:/Users/vikto/Desktop/Pacman-Pipeline/data/raw/sessions/session_12.json")

df = pd.json_normalize(raw["ticks"])

df.dropna()
df.columns


Index(['enemy_momenta', 'enemy_positions_grid', 'enemy_positions_screen',
       'player_buffer', 'player_momentum', 'player_position_grid',
       'player_position_screen', 'score', 'tick'],
      dtype='object')

In [2]:
#read the tick_id and score into a seperate tick table

ticks = df["tick"].values
score = df["score"].values

tick_data = {
    "tick_id" : ticks,
    "score" : score
}

tick_table = pd.DataFrame(data=tick_data)
tick_table.head()


,tick_id,score
0,0,0
1,10,0
2,20,0
3,30,0
4,40,0


In [3]:
#read the player specific data into a seperate player table

df["player_posX"] = df["player_position_screen"].str[0]
df["player_posY"] = df["player_position_screen"].str[1]

player_posX = df["player_posX"]
player_posY = df["player_posY"]


#shift the player momentum buffer by -1 to get the players reaction to the game-state
df["player_buffer"] = df["player_buffer"].shift(-1)

df.drop(df.tail(1).index, inplace=True)

#convert the player momentum buffer into the corresponding bufferIndex
bufferIndex = df["player_buffer"].apply(lambda x: Index[tuple(x)])

#convert the player momentum into the corresponding momentumIndex
momentumIndex = df["player_momentum"].apply(lambda x: Index[tuple(x)])

player_data = {
    "tick" : ticks, 
    "posX" : player_posX,
    "posY" : player_posY, 
    "player_bufferIndex" : bufferIndex, 
    "momentum_Index" : momentumIndex
}

player_table = pd.DataFrame(data=player_data) 

#drop the last row because there is no momentum buffer for it
player_table.drop(player_table.tail(1).index, inplace=True)

player_table.tail()

,tick,posX,posY,player_bufferIndex,momentum_Index
3231,20570,714.0,440.0,2.0,1.0
3232,20580,694.0,440.0,0.0,1.0
3233,20590,688.0,440.0,4.0,2.0
3234,20597,702.0,440.0,4.0,2.0
3235,20600,708.0,440.0,4.0,2.0


In [4]:
#read the enemy specific data into a seperate enemy table

#unnest the momenta and screen position arrays
df_long = df.explode(["enemy_momenta", "enemy_positions_screen"]).reset_index(drop=True)

#fill the player enemy dataframe with player positions to avoid dimension mismatch
df_long[["player_posX", "player_posY"]] = df_long.groupby("tick")[["player_posX", "player_posY"]].ffill()

df_long["player_momentum"] = df_long.groupby("tick")["player_momentum"].ffill()
df_long["player_momentumIndex"] = df_long["player_momentum"].apply(lambda x: Index[tuple(x)])

df_long["enemy_id"] = df_long.groupby("tick").cumcount()

df_long[["enemy_posX", "enemy_posY"]] = pd.DataFrame(df_long["enemy_positions_screen"].tolist(), index=df_long.index)

df_long = df_long.drop(columns=["enemy_positions_screen"])

enemy_id = df_long["enemy_id"].values

#get the opposite Index of the current enem momentum
opposite_directionIndex = df_long["player_momentum"].apply(lambda x: oppositeIndex[tuple(x)])

df_long["momentum_index"] = df_long["enemy_momenta"].apply(lambda x: Index[tuple(x)])
df_long["opposite_direction"] = opposite_directionIndex == df_long["player_momentumIndex"]
df_long["opposite_direction"] = df_long["opposite_direction"].apply(lambda x: int(x))

#distance between enemy and player
df_long["distance"] = np.hypot (
    df_long["enemy_posX"] - df_long["player_posX"],
    df_long["enemy_posY"] - df_long["player_posY"]
)


ticks = df_long["tick"].values

enemy_data = {
    "tick" : ticks, 
    "enemy_id" : enemy_id,
    "momentumIndex" : df_long["momentum_index"],
    "posX" : df_long["enemy_posX"],
    "posY" : df_long["enemy_posY"],
    "player_dist": df_long["distance"],
    "opposite_direction": df_long["opposite_direction"]
}

enemy_table = pd.DataFrame(data=enemy_data)

enemy_table.tail()

,tick,enemy_id,momentumIndex,posX,posY,player_dist,opposite_direction
9703,20597,1,4,760.0,354.0,103.730420,0
9704,20597,2,2,594.0,600.0,193.038856,0
9705,20600,0,1,280.0,440.0,428.000000,0
9706,20600,1,4,760.0,360.0,95.414884,0
9707,20600,2,2,600.0,600.0,193.038856,0


In [5]:
es = ft.EntitySet(id="game_data")
#add the tick table to the entity set
es = es.add_dataframe (
    dataframe=tick_table,
    dataframe_name="ticks",
    index="tick_id",
    make_index=False,
    already_sorted=True
)

#add the player table to the entity set
es = es.add_dataframe (
    dataframe=player_table,
    dataframe_name = "player",
    index="tick_id",
    time_index="tick",
    make_index=True,
    already_sorted=True
)

#add the enemy table to the entity set
es = es.add_dataframe (
    dataframe=enemy_table,
    dataframe_name="enemy",
    index = "tick_id",
    time_index="tick",
    make_index = True,
    already_sorted= True
)

#add the relationships between the tables
es.add_relationship(
    parent_dataframe_name="ticks",
    parent_column_name="tick_id",
    child_dataframe_name="player",
    child_column_name="tick"
)

es.add_relationship(
    parent_dataframe_name="ticks",
    parent_column_name="tick_id",
    child_dataframe_name="enemy",
    child_column_name="tick"
)



Entityset: game_data
  DataFrames:
    ticks [Rows: 3237, Columns: 2]
    player [Rows: 3236, Columns: 6]
    enemy [Rows: 9708, Columns: 8]
  Relationships:
    player.tick -> ticks.tick_id
    enemy.tick -> ticks.tick_id

In [6]:
#default aggregation primitives

default_agg_primitives = ["sum", "max", "min", "mean", "count"]

#deep feature synthesis (DFS)
features = ft.dfs(
    entityset=es, 
    target_dataframe_name="player",
    agg_primitives=default_agg_primitives,
    max_depth=2,
    features_only=True
)

#feature matrix
feature_matrix, feature_names = ft.dfs(
    entityset=es, 
    target_dataframe_name="player",
    agg_primitives=default_agg_primitives,
    max_depth=2,
    features_only=False, 
    verbose=True
)

#pd.options.display.max_columns = 1700
feature_matrix = pd.DataFrame(feature_matrix)
feature_matrix.tail()



Built 48 features
Elapsed: 00:00 | Progress: 100%|██████████


C:\Users\vikto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x0000020A63108B80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
C:\Users\vikto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000020A631096C0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
C:\Users\vikto\AppData\Local\Packages\PythonSoftwareFoundation.Pytho

,tick,posX,posY,player_bufferIndex,momentum_Index,ticks.score,ticks.COUNT(player),ticks.MAX(player.momentum_Index),ticks.MAX(player.player_bufferIndex),ticks.MAX(player.posX),...,ticks.MIN(enemy.opposite_direction),ticks.MIN(enemy.player_dist),ticks.MIN(enemy.posX),ticks.MIN(enemy.posY),ticks.SUM(enemy.enemy_id),ticks.SUM(enemy.momentumIndex),ticks.SUM(enemy.opposite_direction),ticks.SUM(enemy.player_dist),ticks.SUM(enemy.posX),ticks.SUM(enemy.posY)
tick_id,,,,,,,,,,,,,,,,,,,,,
3231,20570,714.0,440.0,2.0,1.0,311,1,1.0,2.0,714.0,...,0.0,147.363496,340.0,300.0,3.0,7.0,0.0,757.744544,1640.0,1340.0
3232,20580,694.0,440.0,0.0,1.0,311,1,1.0,0.0,694.0,...,0.0,136.952547,320.0,320.0,3.0,7.0,0.0,719.653289,1640.0,1360.0
3233,20590,688.0,440.0,4.0,2.0,311,1,2.0,4.0,688.0,...,0.0,123.223374,300.0,340.0,3.0,7.0,0.0,704.262231,1640.0,1380.0
3234,20597,702.0,440.0,4.0,2.0,311,1,2.0,4.0,702.0,...,0.0,103.730420,286.0,354.0,3.0,7.0,0.0,712.769276,1640.0,1394.0
3235,20600,708.0,440.0,4.0,2.0,311,1,2.0,4.0,708.0,...,0.0,95.414884,280.0,360.0,3.0,7.0,0.0,716.453740,1640.0,1400.0
